### Installing requirements and importing the package

In [1]:
# GraphToRDB requirements
# !pip3 install schema
# !pip3 install pyyaml

import GraphToRDB as g2r
import yaml

### Exploring the built-in schema validation

`g2r.validation` returns the `Schemas` object which contains `Schema` objects from the `schema` library.
These `Schema` objects have a `self.validate()` method which takes some data structure as an argument and compares it to the schema shape.

`g2r.validation` has the following `Schema` objects: `data_node`, `data_edge`, and `map`. 

When calling `Schema.validate()`, if the validation fails, a exception will be raised. When it passes, the method will return the input data.

**Example Validation (Pass)**

In [2]:
with open('example_map.yml') as file:
    config_map = yaml.safe_load(file)

g2r.validation.map.validate(config_map)

{'entity_tables': [{'map_node_label': ['Movie'],
   'table_name': 'e_movie',
   'columns': [{'map_node_property': 'tagline',
     'name': 'tagline',
     'dtype': 'str'},
    {'map_node_property': 'title', 'name': 'title', 'dtype': 'string'},
    {'map_node_property': 'released',
     'name': 'release_year',
     'dtype': 'int'}]},
  {'map_node_label': ['Person'],
   'table_name': 'e_person',
   'columns': [{'map_node_property': 'born',
     'name': 'year_of_birth',
     'dtype': 'int'},
    {'map_node_property': 'name', 'name': 'name', 'dtype': 'string'}]}],
 'relationship_tables': [{'map_edge_label': 'ACTED_IN',
   'table_name': 'r_acted_in',
   'map_from_node_label': ['Person'],
   'map_to_node_label': ['Movie'],
   'columns': [{'map_edge_property': 'roles',
     'name': 'roles',
     'dtype': 'list'}]},
  {'map_edge_label': 'DIRECTED',
   'table_name': 'r_directed',
   'map_from_node_label': ['Person'],
   'map_to_node_label': ['Movie']},
  {'map_edge_label': 'PRODUCED',
   'table_

**Example Validation (Fail)**

In [3]:
# Inserting a new key that violates the schema
config_map['new_key'] = None

try:
    g2r.validation.map.validate(config_map)
except Exception as error:
    message = str(error)

print(message)

# remove error
config_map.pop('new_key')

'Map Config (graph to relational)' Wrong key 'new_key' in {'entity_tables': [{'map_node_label': ['Movie'], 'table_name': 'e_movie', 'columns': [{'map_node_property': 'tagline', 'name': 'tagline', 'dtype': 'str'}, {'map_node_property': 'title', 'name': 'title', 'dtype': 'string'}, {'map_node_property': 'released', 'name': 'release_year', 'dtype': 'int'}]}, {'map_node_label': ['Person'], 'table_name': 'e_person', 'columns': [{'map_node_property': 'born', 'name': 'year_of_birth', 'dtype': 'int'}, {'map_node_property': 'name', 'name': 'name', 'dtype': 'string'}]}], 'relationship_tables': [{'map_edge_label': 'ACTED_IN', 'table_name': 'r_acted_in', 'map_from_node_label': ['Person'], 'map_to_node_label': ['Movie'], 'columns': [{'map_edge_property': 'roles', 'name': 'roles', 'dtype': 'list'}]}, {'map_edge_label': 'DIRECTED', 'table_name': 'r_directed', 'map_from_node_label': ['Person'], 'map_to_node_label': ['Movie']}, {'map_edge_label': 'PRODUCED', 'table_name': 'r_produced', 'map_from_node_l

## Using the Transformer Object

In [4]:
print(g2r.Transformer.__doc__)


    Takes
    * data: path to your valid JSON lines file
    * mapping: path to your mapping config YAML or an in-memory dict
    * begin_conversion: default behavior generates RDB format during validation for efficiency. Alternatively, call generate_rdb() after construction.

    Key Constructor Operations
    1. Validates the schema of mappings on construction, will raise error during failure.
    2. Validates data object by object, writing to RDB format for each success (if begin_conversion), will raise error during failure.

    Attributes
    * data: the loaded and validated JSON lines file (list of dicts)
    * mapping: the loaded config (dict)
    * rdb: a dict containing the output relational data

    Output Options
    * in-memory: access a dict of tables (list of lists) from self.rdb
    * csv: call write_to_csv() method
    * sql statements: call generate_sql() method
    


In [5]:
tformer = g2r.Transformer(data='example_data.json', mapping='example_map.yml', begin_conversion=False)

Setting `begin_conversion` to `False` prevents loading data to the RDB format immediately. 
But the `rdb` attribute is still initialized, allowing for a quick visual scan of the table schemas generated by your mapping:

In [6]:
tformer.rdb

{'e_movie': {'header': ['id', 'tagline', 'title', 'release_year'], 'data': []},
 'e_person': {'header': ['id', 'year_of_birth', 'name'], 'data': []},
 'r_acted_in': {'header': ['id', 'from_id', 'to_id', 'roles'], 'data': []},
 'r_directed': {'header': ['id', 'from_id', 'to_id'], 'data': []},
 'r_produced': {'header': ['id', 'from_id', 'to_id'], 'data': []},
 'r_wrote': {'header': ['id', 'from_id', 'to_id'], 'data': []},
 'r_reviewed': {'header': ['id', 'from_id', 'to_id', 'review_text', 'rating'],
  'data': []}}

If you leave `begin_conversion` to the default `True`, the `rdb` attribute will include the converted data (recommended if you're confident in the validity of your schemas). 

Alternatively, call the `generate_rdb()` method after construction to (re)generate the data. This allows for modification of your loaded JSON data before conversion. If you do intend to modify your loaded JSON, it is recommended to re-validate with the objects in the `g2r.validation` object.

> When generating the converted data, if a label is come across that you did not include a mapping for, you will be warned.
>
> In the below example, all instances of "FOLLOWS" trigger such a warning.

In [7]:
tformer.generate_rdb()

/Users/tb/Code/School/DSE/DSE 203/203-graph-to-rdb/GraphToRDB/__init__.py:153: UserWarning: "FOLLOWS" not in mapping. "relationship" of id 239 will be ignored
  warn(f'"{key}" not in mapping. "{element["type"]}" of id {element["id"]} will be ignored')
/Users/tb/Code/School/DSE/DSE 203/203-graph-to-rdb/GraphToRDB/__init__.py:153: UserWarning: "FOLLOWS" not in mapping. "relationship" of id 240 will be ignored
  warn(f'"{key}" not in mapping. "{element["type"]}" of id {element["id"]} will be ignored')
/Users/tb/Code/School/DSE/DSE 203/203-graph-to-rdb/GraphToRDB/__init__.py:153: UserWarning: "FOLLOWS" not in mapping. "relationship" of id 241 will be ignored
  warn(f'"{key}" not in mapping. "{element["type"]}" of id {element["id"]} will be ignored')
/Users/tb/Code/School/DSE/DSE 203/203-graph-to-rdb/GraphToRDB/__init__.py:153: UserWarning: "FOLLOWS" not in mapping. "relationship" of id 497 will be ignored
  warn(f'"{key}" not in mapping. "{element["type"]}" of id {element["id"]} will be ig

Converted data:

In [8]:
tformer.rdb["e_movie"]["data"][:4]

[[0, 'Welcome to the Real World', 'The Matrix', 1999],
 [9, 'Free your mind', 'The Matrix Reloaded', 2003],
 [10,
  'Everything that has a beginning has an end',
  'The Matrix Revolutions',
  2003],
 [11, 'Evil has its winning ways', "The Devil's Advocate", 1997]]

DataFrame compatibility:

In [9]:
from pandas import DataFrame
movies = tformer.rdb["e_movie"]
DataFrame(data=movies["data"], columns=movies["header"])

,id,tagline,title,release_year
0,0,Welcome to the Real World,The Matrix,1999
1,9,Free your mind,The Matrix Reloaded,2003
2,10,Everything that has a beginning has an end,The Matrix Revolutions,2003
3,11,Evil has its winning ways,The Devil's Advocate,1997
4,15,"In the heart of the nation's capital, in a cou...",A Few Good Men,1992
...,...,...,...,...
315,1410,A stiff drink. A little mascara. A lot of nerv...,Charlie Wilson's War,2007
316,1412,This Holiday Season… Believe,The Polar Express,2004
317,1413,Once in a lifetime you get a chance to do some...,A League of Their Own,1992
318,1422,Act like you own the place,Parasite,2019


### Other Outputs

To CSVs:

In [ ]:
tformer.write_to_csv(write_to_directory="transformed_data")

To a PostgreSQL Schema:

In [10]:
# can specify to write to a file instead of returning the text
sql_ddl = tformer.generate_sql()
print(sql_ddl["CREATE"])

CREATE TABLE e_movie (
	tagline		TEXT,
	title		TEXT,
	release_year		INTEGER
);

CREATE TABLE e_person (
	year_of_birth		INTEGER,
	name		TEXT
);

CREATE TABLE r_acted_in (
	from_id		INTEGER,
	to_id		INTEGER,
	roles		TEXT ARRAY
);

CREATE TABLE r_directed (
	from_id		INTEGER,
	to_id		INTEGER
);

CREATE TABLE r_produced (
	from_id		INTEGER,
	to_id		INTEGER
);

CREATE TABLE r_wrote (
	from_id		INTEGER,
	to_id		INTEGER
);

CREATE TABLE r_reviewed (
	from_id		INTEGER,
	to_id		INTEGER,
	review_text		TEXT,
	rating		NUMERIC
);


### TLDR

For simple workflows:
```
import GraphToRDB as g2r

obj = g2r.Transformer(data="my_jsonlines.json", mapping="my_config.yml")
obj.write_to_csv(write_to_directory="my_output_directory")
```

